In [ ]:
import os, glob
import cv2
import mediapipe as mp
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf

# 1) CONFIG
# DATASET_DIR = "File-Path"
IMG_EXTS   = ("*.jpg", "*.png")
MODEL_PATH = "asl_letter_model.keras"
SCALER_PATH = "scaler.pkl"
LE_PATH    = "label_encoder.pkl"

# 2) EXTRACT FEATURES & LABELS
mp_hands = mp.solutions.hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5)
data, labels = [], []

for class_name in sorted(os.listdir(DATASET_DIR)):
    class_dir = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_dir): continue
    for ext in IMG_EXTS:
        for img_path in glob.glob(os.path.join(class_dir, ext)):
            img = cv2.imread(img_path)
            if img is None: continue
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            res = mp_hands.process(rgb)
            if not res.multi_hand_landmarks: continue
            lm = res.multi_hand_landmarks[0].landmark
            feat = []
            for p in lm:
                feat += [p.x, p.y, p.z]
            data.append(feat)
            labels.append(class_name.upper())  # ensure uppercase A–Z

mp_hands.close()
X = np.array(data, dtype=np.float32)          # shape (N,63)
y = np.array(labels)                          # shape (N,)

print(f">> Loaded {len(X)} samples from {len(set(y))} classes")

# 3) ENCODE LABELS & SCALE FEATURES
le = LabelEncoder().fit(y)
y_enc = le.transform(y)                       # 0–26 labels
y_ohe = tf.keras.utils.to_categorical(y_enc)  # one-hot to (N,27)

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# 4) SPLIT
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_ohe, test_size=0.2, random_state=42, stratify=y_ohe)

# 5) BUILD & TRAIN MODEL
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(63,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=32
)

# 6) SAVE EVERYTHING
model.save(MODEL_PATH)
with open(SCALER_PATH, "wb") as f: pickle.dump(scaler, f)
with open(LE_PATH,    "wb") as f: pickle.dump(le,    f)

print("✅ Training complete. Model + scaler + label encoder saved.")


>> Loaded 316 samples from 26 classes
Epoch 1/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.0626 - loss: 3.3279 - val_accuracy: 0.1562 - val_loss: 3.0278
Epoch 2/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1643 - loss: 2.9927 - val_accuracy: 0.1875 - val_loss: 2.7982
Epoch 3/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2270 - loss: 2.8224 - val_accuracy: 0.3281 - val_loss: 2.5975
Epoch 4/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3139 - loss: 2.5577 - val_accuracy: 0.3438 - val_loss: 2.4001
Epoch 5/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3294 - loss: 2.4777 - val_accuracy: 0.4062 - val_loss: 2.1889
Epoch 6/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4595 - loss: 2.1571 - val_accuracy: 0.5000 - val_loss: 1.9844
Epoch 7/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4995 - loss: 2.0691 - val_accuracy: 0.5938 - val_loss: 1.7870
Epoch 8/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5634 - loss: 1.8543 - val_ac